In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator

k = 5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
inputs_dir = '/content/drive/My Drive/BBM409 Assignment1/'

movies = pd.read_csv(inputs_dir + 'movies.csv')
ratings = pd.read_csv(inputs_dir + 'ratings_train.csv')
links = pd.read_csv(inputs_dir + 'links.csv')
tags = pd.read_csv(inputs_dir + 'tags.csv')
tests = pd.read_csv(inputs_dir + 'ratings_test_without_gt_new.csv')

In [ ]:
def find_rating_pivot():
  user_ratings = ratings.groupby("userId")['rating'].count()
  movie_ratings = ratings.groupby('movieId')['rating'].count()
  
  # ratings_per_user.hist(bins=20, range=(0,500))
  # ratings_per_movie.hist(bins=20, range=(0,500))

  movie_ratings_df = pd.DataFrame(movie_ratings)
  top_movies = movie_ratings_df.index.tolist()

  user_ratings_df = pd.DataFrame(user_ratings)
  prolific_users = user_ratings_df.index.tolist()

  ratings_with_filter = ratings[ratings.movieId.isin(top_movies)]
  ratings_with_filter = ratings[ratings.userId.isin(prolific_users)]


  rating_matrix = ratings_with_filter.pivot_table(index='userId', columns='movieId', values='rating')
  # replace NaN values with 0
  rating_matrix = rating_matrix.fillna(0)
  dict_rating_matrix  = rating_matrix.to_dict('index')

  return rating_matrix, dict_rating_matrix

rating_matrix, dict_rating_matrix = find_rating_pivot()

In [ ]:
rating_matrix.head()


movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184641,184721,184791,184997,185029,185031,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188751,189043,189111,189333,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,4.0,0.0,3.0,0.0,4.0,4.0,4.0,0.0,2.0,0.0,2.0,5.0,0.0,4.0,3.0,4.0,3.0,0.0,0.0,0.0,3.0,4.0,4.0,5.0,0.0,0.0,0.0,4.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def pearson(list1, list2):
    return np.corrcoef(list1, list2)[0][1]

def cos_sim(list1, list2):
  a = np.dot(list1, np.transpose(list2)) / ((np.linalg.norm(list1)) * (np.linalg.norm(list2)))
  
  return [i*10 for i in a] 

In [ ]:
def similar_users(userId, matrix):
    # create a df for intended current user
    user = matrix[matrix.index == userId]
    # print(user.shape)
    other_users = matrix[matrix.index != userId]
    # other_users = other_users.transpose()
    # calculate cos similarity between user and each other user
  
    # from sklearn.metrics.pairwise import cosine_similarity
    # similarities = cosine_similarity(user,other_users)[0]
    similarities = cos_sim(user,other_users)[0]

    indices = other_users.index.tolist()
    
    index_similarity = dict(zip(indices, similarities))
    
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    
    return users, top_users_similarities

In [ ]:
def find_case(user_id, movie_id, rating):
  case =0;
  #if user id doesnt exist in train datas
  if(user_id not in dict_rating_matrix.keys()):
    case = 1
    return case
  #if rating doesnt exist in train datas
  if(movie_id not in dict_rating_matrix.get(user_id).keys()):
    case = 2
  return case;

In [ ]:
def find_rating_by_userid(user_id, movie_id):
  return rating_matrix.loc[(user_id, movie_id)]

In [ ]:
def predict(weighted_or_not, similar_user_indices, similar_user_indices_and_similarities, movieId):
  rating = 0;
  total=0;
  if(weighted_or_not == False):
    for idx in similar_user_indices:
      rate_by_user = find_rating_by_userid(idx, movieId)
      if (rate_by_user!=0):
        total += rate_by_user
      rating = total/k
  elif (weighted_or_not == True):
    for (userId, sim) in similar_user_indices_and_similarities:
      rate_by_user = find_rating_by_userid(userId, movieId)
      total += rate_by_user*sim
      rating = total
  if rating >= 5.01:
    rating = 5
  return rating;

In [ ]:
pred_results = []

def kfold(train_ratings_main, k2, weighted_or_not):
  data_size = len(train_ratings_main.index)
  # print("data_size", data_size)
  test_size = int(data_size/k2)
  # print("test_size", test_size)
  results =[]
  
  start_idx = 0;
  
  for i in range(k2):
    MAE = 0
    # print("-------test---------")
    test_ratings_splitted = train_ratings_main
    test_ratings_splitted = test_ratings_splitted.drop([i for i in range(0,start_idx)])
    test_ratings_splitted = test_ratings_splitted.drop([i for i in range(start_idx + test_size , data_size)])
    # print(test_ratings_splitted.head())

    # print("-------train---------")
    train_ratings_splitted = train_ratings_main
    train_ratings_splitted = train_ratings_splitted.drop([i for i in range(start_idx , start_idx + test_size)])
    # print(train_ratings_splitted.head())

    
    pred_results_in = []
    for index, row in test_ratings_splitted.iterrows():
      user_id_input = int(row['userId'])
      movie_id_input = int(row['movieId'])
      rating_input = row['rating']
      
      sim_user_indices, sim_user_indices_and_similarities = similar_users(user_id_input, rating_matrix)
      # case = find_case(user_id_input, movie_id_input, rating_input)
      
      
      prediction = predict( weighted_or_not, sim_user_indices, sim_user_indices_and_similarities, movie_id_input )
      
      pred_results_in.append([prediction, rating_input])
      # print("prediction: {0:.1f} ----- rating: {1:.1f}".format(prediction, rating_input))
      
      MAE += abs(rating_input - prediction)
    pred_results.append(pred_results_in)
    print("MAE/test_size: " , MAE/test_size)
    
    results.append( MAE/test_size)
    start_idx = start_idx + test_size

  return results


In [ ]:
print(kfold(ratings, 5, True))

In [ ]:
def recommend_item(user_index, sim_user_indices, matrix, items=5):
    
    sim_users = matrix[matrix.index.isin(sim_user_indices)]
    sim_users = sim_users.mean(axis=0)
    sim_users_df = pd.DataFrame(sim_users, columns=['mean'])
    
    
    user_df = matrix[matrix.index == user_index]
    user_df_transposed = user_df.transpose()
    user_df_transposed.columns = ['rating']
    user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]

    movies_unseen = user_df_transposed.index.tolist()
    
    sim_users_df_filtered = sim_users_df[sim_users_df.index.isin(movies_unseen)]
    sim_users_df_ordered = sim_users_df.sort_values(by=['mean'], ascending=False)
    topn_movie = sim_users_df_ordered.head(items)
    topn_movie_indices = topn_movie.index.tolist()
    movie_info = movies[movies['movieId'].isin(topn_movie_indices)]
    
    return movie_info

In [ ]:
recommend_item(227, similar_user_indices, rating_matrix)

,movieId,title,genres
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
828,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller
900,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
902,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
